In [ ]:
!pip install sentencepiece 

In [2]:
from transformers import AutoTokenizer 
import datasets 
from loguru import logger
import tqdm
import torch

In [3]:

tokenizer = AutoTokenizer.from_pretrained("Konstantinos/el_llama_3bn")

tokenizer.encode("[INST]"), tokenizer.encode("[/INST]"), tokenizer.encode("<<SYS>>"), tokenizer.encode("<</SYS>>") 



You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


([0, 4], [0, 5], [0, 6], [0, 7])

In [4]:
tokenizer.encode("hey  [INST]", add_special_tokens=True), tokenizer.decode(4)

([0, 2846, 5418, 21794, 4], '[INST]')

In [5]:
data = datasets.load_dataset("allenai/c4", "el", split="train", streaming=True)

Resolving data files:   0%|          | 0/1024 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1024 [00:00<?, ?it/s]

In [6]:
seed_for_shuffle = 42 
    
logger.info(f"Shuffling data with seed {seed_for_shuffle}")
data: datasets.Dataset = data.shuffle(seed=seed_for_shuffle)


2024-04-07 10:32:45.776 | INFO     | __main__:<module>:3 - Shuffling data with seed 42


In [7]:
500_000_00

50000000

In [8]:
all_tokens = []

tokens_so_far = 0 
docs = 0 
with tqdm.tqdm() as pbar: 
    for idx, example in enumerate(data):
        text = example['text']
        tokens = tokenizer.encode(text + '</s>')
        
        tokens_so_far += len(tokens)
        all_tokens.extend(tokens)
        if idx % 10000 == 0 and idx!=0: 
            pbar.set_description( f"Docs: {idx}, tokens seen {tokens_so_far}")
            docs+=10000
            pbar.update(docs)

        #if tokens_so_far>100_000_000:
        #    break 
        if tokens_so_far > 1_000_000_000:
            break

Docs: 930000, tokens seen 989659684: : 43710000it [41:50, 17408.90it/s]


In [18]:
tokenizer.decode(all_tokens[0:100])

'<s> ΗΜΕΡΟΛΟΓΙΟ ΚΑTΑΣTΡΩΜΑTΟΣ: ΓΙΟΡΤΗ και ΓΕΝΕΘΛΙΑ\nTα δύο τραγούδια και το video με τον Σαρλώ είναι αφιερωμένα με αγάπη στα κορίτσια του σκακιστικού τμήματος Μυρτώ και Κατερίνα, για τη γιορτή της πρώτης και τα γενέθλια της δεύτερης σήμερα 24 του Σεπτέμβρη, ευχόμενός τους'

In [9]:
to =  int(0.9*len(all_tokens))
to

900000154

In [10]:
torch.save(all_tokens[0:to], "./data/training.pt")


In [11]:
torch.save(all_tokens[to:], "./data/validation.pt")

In [ ]:
torch.save( all_tokens[0:100000], "./data/small.pt")

In [ ]:
len(all_tokens[to:])

In [ ]:
37_499_661

In [ ]:
!ls -lah ./dataa

In [13]:

torch.save(all_tokens[0:100000], "./data/training_small.pt")
torch.save(all_tokens[100_000:11000], "./data/validation_small.pt")

